## [pyKNEEr](https://github.com/sbonaretti/pyKNEEr)

# [Preprocessing MR Knee Images](https://sbonaretti.github.io/pyKNEEr/preprocessing.html)

Preprocessing of MR images is fundamental to standardize spatial information and intensity of images

In the pipeline below:
- Steps 1-3 uniform image orientation, knee laterality, and image origin. The resulting image has the same intensity as the acquired `.dcm` image, but different collocation in space. It is saved as `_orig.mha`
- Steps 4-6 act on the intensities of the image, by correcting the inhomogeneous magnetic field, rescaling to a fixed range, and enhancing the cartilage contours [1]. The resulting image has different intensity than the acquired `.dcm` image, and it is used for atlas-based segmentation. It is saved as `_prep.mha`. These steps are computationally expensive, so if not needed, avoid them by setting `intensity_standardization` to zero (see below) 

## Import packages

In [ ]:
from pykneer import pykneer_io as io
from pykneer import preprocessing_for_nb as prep

## Image information

Inputs:   
- `input_file_name` contains the list of the images to be preprocessed 
- `nOfCores` is the number of cores used for computations (computations are parallelized for speed)
- `intensity_standardization` is a flag that determines if the intensity standardization is performed (0=no, 1=yes)

In [ ]:
input_file_name           = "./image_list_preprocessing.txt"
n_of_cores                = 1 # change the number of cores according to your computer
intensity_standardization = 1

#### Read image data
- `imageData` is a dictionary (or struct), where each cell corresponds to an image. For each image, information such as paths and file names are stored  

In [ ]:
image_data = io.load_image_data_preprocessing(input_file_name)

## Spatial standardization

#### Step 0: Read the dicom images

In [ ]:
prep.read_dicom_stack(image_data, n_of_cores)

#####   Save image header
The image header of the first slice of the dicom stack is saved in a `.txt` file. This can be useful to extract information such as subject's date of birth, exam date, etc.

In [ ]:
prep.print_dicom_header(image_data, n_of_cores)

#### Step 1: Change orientation to RAI 
All images are oriented to the RAI (right, anterior, inferior) orientation

In [ ]:
prep.orientation_to_rai(image_data, n_of_cores)

#### Step 2: Change laterality if knee is right
Right knees are flipped to left knees to have the same laterality 

In [ ]:
prep.flip_rl(image_data, n_of_cores)

#### 3. Set image origin to (0,0,0)
All images are set to the origin (0,0,0). Images are saved as *_orig.mha and they are anonymized

In [ ]:
prep.origin_to_zero(image_data, n_of_cores)

## Intensity standardization

#### 4. Correct magnetic field inhomogeneities
Magnetic fields inhomogeneities create grey shades on images. This correction removes these shades. This is the longest step of the processing. It can take up to 15-20 min on a standard PC or laptop

In [ ]:
if intensity_standardization == 1:
    prep.field_correction(image_data, n_of_cores) 

#### 5. Rescale intensities to [0 100]
Intensies are rescaled to the fixed range [0,100] to obtain homogeneous intensity range across images

In [ ]:
if intensity_standardization == 1:
    prep.rescale_to_range(image_data, n_of_cores) 

#### 6. Edge preserving smoothing
Cartilage contours are enhanced using a curvature anisotropic diffusion filter

In [ ]:
if intensity_standardization == 1:
    prep.edge_preserving_smoothing(image_data, n_of_cores) 

## Visualize original and preprocessed image
Set the variable `view_modality` to `0` to show one slice of the image (static and faster rendering) or to `1` to scroll though the image (interactive and slower rendering). Note that when using interactive rendering, images might NOT be saved  for when reopening the notebook.

Each row corresponds to one image. Left: Original image after spatial preprocessing (steps 1-3). Right: Preprocessed image after intensities correction (steps 4-6).  
*Note*: A total black rendering of an original image is caused by very large intensity distribution.

In [ ]:
view_modality = 0; # 0 for static, 1 for interactive
fig = prep.show_preprocessed_images(image_data, intensity_standardization, view_modality);
display(fig)

---

### References

[1] Shan L., Zach C., Charles C., Niethammer M. [*Automatic Atlas-Based Three-Label Cartilage Segmentation from MR Knee Images.*](https://www.ncbi.nlm.nih.gov/pubmed/25128683) Med Image Anal. Oct;18(7):1233-46. 2014.

### Dependencies

In [ ]:
%load_ext watermark
%watermark -v -m -p pykneer,SimpleITK,itk,numpy,matplotlib,multiprocessing,ipywidgets,watermark
print (" ")
%watermark -u -n -t -z 